# Introduction


## What is RAG?

**Retriever augmented generation** (**RAG**) is a system that improves the response generated by a LLM in two ways:

* First, the information is retrieved from a dataset that is stored in vector database; the query is used to perform similarity search in the documents stored in the vector database.
* Second, by restraining the context provided to the LLM to content that is similar with the initial query, stored in the vector database, we can reduce significantly (or even eliminate) LLM's halucinations, since the answer is provided from the context of the stored documents.

An important advantage of this approach is that we do not need to fine-tune the LLM with our custom data; instead, the data is ingested (cleaned, transformed, chunked, and indexed in the vector database).

## Procedure

* We start by using directly a prompt with questions for the LLM using a transformers pipeline.
* We test the pipeline with few questions.
* We then ingest the text of the Letters of Ion Ghica in a vector database that will serve as the retriever. The vector database used is ChromaDB.
* We assemble the retriever-generation system. For this, we compose a prompt with the system message instructing the LLM how to use the initial query and the context retrieved from the vector database; in the user message we give the query as input.
* We test with the same questions we tried without the RAG system (directly prompting the LLM) and more questions.

# Install and include packages

In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install langchain
!pip install sentence-transformers
!pip install chromadb

Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the 

In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.5 MB/s eta 0:00:00


In [3]:
from time import time
from IPython.display import display, Markdown
from transformers import pipeline

from transformers import AutoTokenizer, AutoModelForCausalLM

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

2024-06-18 08:21:18.470294: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 08:21:18.470395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 08:21:18.603237: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Prepare the model pipeline

In [4]:
pipe = pipeline("text-generation", 
                model="/kaggle/input/openllm-ro-q4b/transformers/7b-instruct-q4b/1/RoLlama2-7b-Instruct-v1-q4b",
               kwargs=['_load_in_4bit'])

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Test the text generator function

In [5]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipe.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipe(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipe.model.config.eos_token_id
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime


system_message = """
You are an AI assistant designed to answer simple questions from the works of Ion Ghica.
Please restrict your answer to the exact question asked.
"""

# Run few tests with the text generator

In [6]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [7]:
response = query_model(
    system_message,
    user_message="Cine a fost dascalul Chiosea?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.




**<font color='red'>Question:</font>** Cine a fost dascalul Chiosea? 

**<font color='green'>Answer:</font>**  Dascalul Chiosea a fost un dascal (profesor) din Chiosea, o comună din județul Prahova, România.  

**<font color='magenta'>Total time:</font>** 4.91 sec.

In [8]:
response = query_model(
    system_message,
    user_message="Cine a fost Marin Serghiescu?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cine a fost Marin Serghiescu? 

**<font color='green'>Answer:</font>**  Marin Serghiescu a fost un politician român, membru al Partidului Conservator, care a deținut funcția de ministru de interne în timpul domniei lui Alexandru Ioan Cuza.  

**<font color='magenta'>Total time:</font>** 4.78 sec.

In [9]:
response = query_model(
    system_message,
    user_message="Cum il chema pe fratele lui Teodor Diamant?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cum il chema pe fratele lui Teodor Diamant? 

**<font color='green'>Answer:</font>**  Teodor Diamant a avut un frate pe nume Alexandru Diamant.  

**<font color='magenta'>Total time:</font>** 2.1 sec.

In [10]:
response = query_model(
    system_message,
    user_message="La cine pranzaste Ion Ghica pe timpul calatoriei sale la Iasi?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** La cine pranzaste Ion Ghica pe timpul calatoriei sale la Iasi? 

**<font color='green'>Answer:</font>**  Ion Ghica a luat masa cu Ion C. Brătianu în timpul călătoriei sale la Iași.  

**<font color='magenta'>Total time:</font>** 3.41 sec.

In [11]:
response = query_model(
    system_message,
    user_message="Unde s-a născut Bărzof?",
     temperature=0.1,
     max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Unde s-a născut Bărzof? 

**<font color='green'>Answer:</font>**  Bărzof s-a născut în satul Bărzău din județul Buzău, România.  

**<font color='magenta'>Total time:</font>** 3.23 sec.

In [12]:
response = query_model(
    system_message,
    user_message="Cât de mare era curtea banului Ghica?",
     temperature=0.1,
     max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cât de mare era curtea banului Ghica? 

**<font color='green'>Answer:</font>**  Curtea banului Ghica era de aproximativ 100 de hectare.  

**<font color='magenta'>Total time:</font>** 2.16 sec.

The answers are somehow accurate, with some exceptions. Let's check if we can get better answers with a RAG system where we index the text of the letters in a vector database that we will use as the retriever part of the system.

# Retrieval preparation

We will create now the retrieval functionality. We will use a ChromaDB instance for vector database. We will ingest the text from Ion Ghica work, "Letters to Vasile Alecsandri" in Romanian. Here we have the comlete text of Ion Ghica most known work.

## Ingest the text

### Load the document

In [13]:
loader = PyPDFLoader("/kaggle/input/scrisori-ctre-vasile-alecsandri-de-ion-ghica/Scrisori_catre_Vasile_Alecsandri.pdf")
documents = loader.load()

### Split the text and prepare the embeddings

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800, 
            chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)
# create a vectorstore database
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Ingest the data in ChromaDB

In [15]:
vector_db = Chroma.from_documents(documents=all_splits, 
                                embedding=embeddings, 
                                persist_directory="chroma_db")

### Define the retriever

In [16]:
retriever = vector_db.as_retriever()

## Define the query

We define functions to generate the system prompt, user prompt and to get the context documents.

In [17]:
def get_system_prompt(document_text):
    system_prompt=f"""
        INSTRUCTIUNI:
            Vă rog să răspundeți la întrebările utilizatorilor folosind DOCUMENT-ul furnizat.
            Limitați răspunsul la faptele consemnate în DOCUMENT.
            Dacă documentul nu conține faptele pentru a răspunde la ÎNTREBARE menționați aceasta 
            în mod clar și nu returnați nimic.
        DOCUMENT:
            {document_text}
        """
    return system_prompt

In [18]:
def get_user_prompt(query):
    user_prompt =f"""
                ÎNTREBARE:
                    {query}
                RĂSPUNS:
                """
    return user_prompt

In [19]:
def get_documents(query):
    # generate the answer
    docs = retriever.get_relevant_documents(query)
    data = ""
    for item in list(docs):
        data += item.page_content
    
    return data

We define a colorized text function for RAG.

In [20]:
def colorize_text_rag(text):
    for word, color in zip(["Question", "Prompt", "Answer", "Total time", "Context"], 
                           ["blue", "magenta", "red", "darkblue", "green"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [21]:
template = "\n\nQuestion:\n{question}\n\nPrompt:\n{prompt}\n\nAnswer:\n{answer}\n\nTotal time:\n{total_time}\n\nContext:\n{context}"

def rag_query(query,
         temperature=0.1,
         max_length=1024,
         show_context=False):
    start_time = time()
    
    # Retriever
    context = get_documents(query)
    
    # Augmented generation
    system_message = get_system_prompt(context)
    user_message = get_user_prompt(query)

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipe.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipe(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipe.model.config.eos_token_id
    )

    answer = sequences[0]['generated_text']
    end_time = time()
    total_time = f"{round(end_time-start_time, 2)} sec."
    if show_context:
        return template.format(question=query, prompt=user_message, answer=answer, total_time=total_time,context=context)
    else:
        return template.format(question=query, prompt=user_message, answer=answer, total_time=total_time,context=None)


# Perform tests with RAG

In [22]:
response = rag_query("Cum il chema pe fratele lui Theodor Diamant?",
     temperature=0.1,
     max_length=64)
display(Markdown(colorize_text_rag(response)))

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




**<font color='blue'>Question:</font>**
Cum il chema pe fratele lui Theodor Diamant?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Cum il chema pe fratele lui Theodor Diamant?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Nicolae Alexandrescu 

**<font color='darkblue'>Total time:</font>**
2.24 sec.

**<font color='green'>Context:</font>**
None

In [23]:
response = rag_query("Cum il chema pe fratele lui Theodor Diamant?",
     temperature=0.1,
     max_length=64,
    show_context=True)
display(Markdown(colorize_text_rag(response)))



**<font color='blue'>Question:</font>**
Cum il chema pe fratele lui Theodor Diamant?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Cum il chema pe fratele lui Theodor Diamant?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Nicolae Alexandrescu 

**<font color='darkblue'>Total time:</font>**
2.08 sec.

**<font color='green'>Context:</font>**
Pe Nicolae Alexandrescu, cum l-a auzit Grigorie Ghica, l-a
luat în casă, a-l plimba cu dânsul noaptea în butcă dinainte,l'empereur Alexandre et l'empereur Nicolas avait rendu si
riche et si prospere, votre administration l'a reduit a la plus
complete misere: il arrive tous les jours a Petersbour g des
plaintes et des lamentations de la part des habitants d'Ismail
et de Bolgrad'.
Am raportat atunci aceste cuvinte domnitorului Cuza,
zicându-i că ele îmi făcuseră mie mare impresiune și că mi
se păreau un rău prognostic pentru viitorul Basarabiei.Cîmpineanu, logofătul Nicolae Dudescu, era de a pregăti
națiunea din vreme, ca acea desfășurare de evenimente carecînd este vorba de o societate puțin dezvoltată, ai cărei
membri nu sunt încă în stare să judece și să prețuiască ceea

In [24]:
response = rag_query("Cine a fost dascalul Chiosea?",
     temperature=0.1,
     max_length=64,
    show_context=True)
display(Markdown(colorize_text_rag(response)))



**<font color='blue'>Question:</font>**
Cine a fost dascalul Chiosea?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Cine a fost dascalul Chiosea?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Dascalul Chiosea a fost un dascal roman care a trait in secolul al 18-lea. El a fost un dascal roman care a trait in secolul al 18-lea. El a fost un dascal roman care a trait in secolul al 

**<font color='darkblue'>Total time:</font>**
7.65 sec.

**<font color='green'>Context:</font>**
69ceasornicul deschis pe catedră ca nu cumva minutarul să
treacă peste semn.
La Chiosea, ca la Chiriță, ca la Stan, lucrurile mer geau
altfel; viața lor era cu copiii din școală, nu știa nici șosea,
nici cafe-șantan, nici deputăție. Dimineața buchille, Oftoicu
și Psaltirea, după prânz psaltichia. Chiosea avea un pa, vu,
ga, di care-i ieșea pe nas cale de o poștă.
De pe la de-alde Chiosea ieșeau dieci de visterie și
calemgii; la de-alde el au învățat să scrie românește
logofătul Greceanu, Văcăreștii, Anton Pan, Nănescu, Paris
Momuleanu etc.
Acei bieți dăscălași, cari au fost depozitari limbei și
naționalității noastre, duceau o viață zdruncinată, plină de
privațiuni și de coate-sparte, fără bene-merenti și fără
recompense naționale, fără pensii și parapensii; plătiți ca vaireprezenta, că puii corbului erau nevinovații de români, și
lebăda, omul care dă sfaturi bune, și ceruse pedeapsa
cutezătorului autor:
 
A zis ...asemenea nebunie, acela ar fi numai vrednic de plîns, iar nu
de temut. Folosiți-vă de binefacerile ce societatea noastră285unul din ucenicii săi; a trăit mult cu dânsul la Munich și la
Paris.
Nu am cunoscut un om mai blând, mai modest și mai
devotat decât dânsul. Filantrop și umanitar , era din neamul
acelora din cari au ieșit apostolii, martirii și sfinții.
O fi având el, oare, o piatră pe mormânt?

In [25]:
response = rag_query("Cine l-a luat pe Marin Serghiescu cântăreț la Biserica Negustorilor?",
     temperature=0.1,
     max_length=64,
    show_context=True)
display(Markdown(colorize_text_rag(response)))



**<font color='blue'>Question:</font>**
Cine l-a luat pe Marin Serghiescu cântăreț la Biserica Negustorilor?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Cine l-a luat pe Marin Serghiescu cântăreț la Biserica Negustorilor?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Alecaci V ilara 

**<font color='darkblue'>Total time:</font>**
3.97 sec.

**<font color='green'>Context:</font>**
Acea scriere a lui Bălcescu a fost foarte apreciată de cititorii
dup-atunci și a avut chiar onorurile unei edițiuni separate,
tipărită tot în tipografia lui Kogălniceanu.
Mai târziu, pe la anii 1844, 1845 și 1846, el a publicat mai
multe lucrări istorice în revista intitulată Magazinul istoric,
ce redacta în colaborare cu eruditul profesor T reboniu
Laurianu. Într -acea colecțiune se găsește scrierea asupra
familiei Cantacuzineștilor , după documente ce-i
încredințase generalul Rudolf Cantacuzino din Rusia.
Nicu Bălcescu scria lesne, stilul său era limpede, strâns,
nervos și elegant, precum îl vedem în Istoria lui Mihai
Viteazul și în Istoria muncitorilor plugari din România;
adeseori înflorit și poetic, precum îl găsim în traducerea
făcută de el a Cântării României.472lumea universitară și patronată în România de consulatul
rusesc.
Din punctul lor de vedere, grecii aveau cel mai mare interes
să urmărească realizarea acelui scop, căci la ei învățătura
era răspîndită în toate clasele, pe cînd celealalte populațiuni
nici nu se gîndeau încă la înființarea de alte școli decît cele
grecești. Clerul ortodox pretutindeni în imperiu era grec,
avînd în mînile sale toate averile bisericești; grecii erau
poporul cel mai luminat și mai avut din Orient. Simțul
național era la ei dezvoltat în cel mai mare grad, pe cînd
celealalte popoare ale Orientului confundau încă
naționalitatea cu biserica, încît bulgarii, arnăuții, românii
din Macedonia se credeau greci, fiindcă biserica lor se
numea biserica ritului oriental grec.73de-i cânta cântece de lume. Când Grigorie Ghica s-a făcut
domn, Nicolae Alexandrescu a ajuns cafegi-bașa.
Pe Marin Ser ghiescu l-a luat Alecache V illara cântăreț la
Biserica Negustorilor , l-a primblat pe dânsul pe la
Petersbur g, cu Regulamentul; dar mai pe urmă, molipsindu-
se de boala patriotismului, a fost arestat cu Mitică Filipescu,
cu Nicolae Bălcescu și cu T elegescu, trimis la ocnă, unde
și-a spășit păcatele șapte ani, până când a venit de l-a scos
revoluția din patruzeci și opt, ca să-i procure dulcile fericiri
ale unei proscripțiuni de zece ani. El a fost cunoscut mai
mult sub numele de Marin Naționalu. V odă Cuza îl iubea, îl
avea adesea la masă și-l mângâia cu numele de moș Marin.
Când a venit în București dascălul Lazăr , băieții de laforței.
„Părinții bisericei — zice Cantu — proclamau libertatea
credințelor în tot timpul persecuțiunii religiei creștine,
îndată însă ce a încetat acea persecuțiune și biserica a
devenit autoritate, au început la rîndul lor a vorbi în numele
puterii.” Aceste două curente ale libertății și al autorității le
găsim în luptă în toate societățile; și chiar în cele mai
înaintate în civilizațiune găsim o luptă neîncetată între
justiție și putere, fiecare posedînd o acțiune puternică
asupra societății lor și biruind cînd una, cînd alta.

In [26]:
response = rag_query("Din cine era coborâtor Ioniță Sandulache Sturdza?",
     temperature=0.1,
     max_length=64,
    show_context=True)
display(Markdown(colorize_text_rag(response)))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset




**<font color='blue'>Question:</font>**
Din cine era coborâtor Ioniță Sandulache Sturdza?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Din cine era coborâtor Ioniță Sandulache Sturdza?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Ioniță Sandulache Sturdza era un descendent al lui Vlad Țepeș. 

**<font color='darkblue'>Total time:</font>**
4.17 sec.

**<font color='green'>Context:</font>**
cunoască păsurile țării, cerea să se trimită la Constantinopol
o deputațiune cu care guvernul otoman să se poată înțelege.
În deputațiunile trimise de la Iași și de la București era și
Ioniță Sandulache Sturdza, un coborâtor al lui Vlad Țepeș și
Grigorie Dimitrie Ghica, nepot de frate decăpitatului
Grigorie Alexandru Ghica al Moldovei.
După mai multe conferințe cu vizirul și cu reis-efendi,
conferințe cari au durat mai bine de două luni, pe la iulie
sus-menționații doi boieri au fost anunțați că sunt numiți38Când eram pe la 1829 la Craiova, l-am văzut de gât cu
generalul Roth, juca cărți cu miile de galbeni, căci era peste
proviant.asupra lui T udor, îi pun în fiare mâinile și picioarele și-l duc
în tabăra lui Ipsilante, unde-l aștepta Caravia, Cavaleropolo344Scrisori cătr e Vasile Alecsandri  de Ion Ghica  
Legile
Londra, 1 noiembre 1884
Iubite amice,
Puterea, ca să fie suferită și respectată, trebuie să emane de
la națiune, să-și aibă rădăcinile ei înfipte în spiritul și
trebuințele poporului.
Cînd principiul de la care purcede a fost cucerirea, violența
sau spiritul militar , guvernul acela n-o poate duce decît
timpul cît principiul care l-a impus se mănține întreg,
pășind din victorie în victorie și din concistă în concistă;
îndată însă ce i-o lipsi acel prestigu, slăbește din zi în zi,
devine din ce în ce mai arbitrar și mai tiranic, se corumpe și
cade în descompunere, dovadă starea provinciilor romane în
timpul decadenței; dovadă guvernele provinciilor
Imperiului otoman.
Esistența și durata unui guvern devine numai o cestiune de

In [27]:
response = rag_query("Unde a crescut Bârzof?",
     temperature=0.1,
     max_length=64,
    show_context=True)
display(Markdown(colorize_text_rag(response)))



**<font color='blue'>Question:</font>**
Unde a crescut Bârzof?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Unde a crescut Bârzof?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Bârzof a crescut în Bugeac. 

**<font color='darkblue'>Total time:</font>**
3.71 sec.

**<font color='green'>Context:</font>**
383Scumpul meu amic,
Auzit-ai de unul Bârzof? Dacă n-ai auzit, eu să-ți spui cine
și ce era; căci l-am văzut și l-am auzit cu ochii și cu urechile
mele.
Bârzof era, cum zice românul, un om cât un munte: înalt,
gros și chipeș, ochi mititei în fundul capului, umerii
obrazului scoși și nasul numai o fărâmiță de sfârc: tipul de
mongol cel mai bine caracterizat. După celealalte toate, era
și ciupit de vărsat, și ciupit așa de adânc, că în fiecare
gropiță se putea ascunde bobul de mazăre.
Născut în Bugeag, fusese însă crescut printre românii de pe
malul drept al Bugului, unde tată-său a fost ani mulți
cinovnic la spirtoase și vorbea românește ca apa. Intrat de
copil ca piser în cancelaria poliției, ajunsese nacealnic și
dobândise cinul de calejki savetnik, încât, când îi vorbeai,38Când eram pe la 1829 la Craiova, l-am văzut de gât cu
generalul Roth, juca cărți cu miile de galbeni, căci era peste
proviant.Pe Nicolae Alexandrescu, cum l-a auzit Grigorie Ghica, l-a
luat în casă, a-l plimba cu dânsul noaptea în butcă dinainte,277Pe cel dintâi îl chema Barbu și-i zicea Bărbucică, pe celalalt
Teodor , Tudorachi. T atăl lor era unul Diamandi Mehtupciu,
nepot sau strănepot de-ai sfântului Dimitrie de la Mitropolia
din București. Era sau nu era rudă cu cuviosul, Dumnezeu
știe; adevărul este că tatăl, moșul sau strămoșul lui Iamandi
venise în țară de peste Dunăre cu moaștele făcătorului de
minuni; îi zicea Mehtupciu pentru că fusese scriitor de
limba turcească la postelnicie. Fiul său T udorache adoptase
numele de T eodor Diamant, sau mai scurt, Diamant.
Pe Bărbucică l-am cunoscut. Știi că nici natura, nici
apucăturile nu-l împingeau la carte; n-a învățat nici să scrie,
nici să citească, era cam idiot. Îi zicea Bălbuitu din cauza
anevoinței ce avea la vorbire; nu era cu desăvârșire surd,

In [28]:
response = rag_query("Cât de mare era curtea banului Ghica?",
     temperature=0.1,
     max_length=64,
    show_context=True)
display(Markdown(colorize_text_rag(response)))



**<font color='blue'>Question:</font>**
Cât de mare era curtea banului Ghica?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Cât de mare era curtea banului Ghica?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Curtea banului Ghica era cât o moșie de mare. 

**<font color='darkblue'>Total time:</font>**
4.11 sec.

**<font color='green'>Context:</font>**
364mai închine, că și nemții, și muscalii, dacă băteau pe turci,
tot pe turta lor trăgeau spuza.
La cererea d-a se da țării domn pământean, pe bătrânul
Părscoveanu, Poarta răspunsese trimițând pe un alt fanariot
în locul lui Alexandru Moruz, pe Mihai Suțu.
În seara de Mucenici, boierii cei mai colțați erau adunați la
sfat la banul Dumitrache Ghica, fratele lui Grigorie-vodă,
asasinat pentru împotrivirea ce făcuse la darea Bucovinei.
Erau adunați la banul bătrânul, cum îi ziceau ca să-l
deosebească de doi fii ai săi, Costache și Scarlat, cari
amândoi ajunseseră tot bani mari și ei.
Curtea banului Ghica era cât o moșie de mare; începea din
capătul despre apus al Podului Gârlei, devale de Zlătari,
mergea în jos pe malul apei până în dreptul caselormai bănoasă, era de-a ataca caravana de la Mecca și a jăfui
cămila care ducea darurile sultanului la mormîntul
profetului.367ani cu luntrea prin mahalaua Izvorul, pe strada Brezoianu,
prin Dudescu, pe la Antim și pe dinaintea bisericei Doamna
Bălașa? Astăzi, grădina, curtea, paraclisul și casa banului
Scarlat Ghica s-au împărțit în bucățele pătrate și
numerotate, s-au tras printre dânsele strade și s-au clădit
case și căscioare.
În sfaturile boierilor , numele eroului de la Marengo era de
câtăva vreme foarte adesea pomenit; speranțele tuturor erau
întoarse către Franța și către minunile ce se auzea că se
petreceau acolo. Deși unii dintre boieri, ca Pană Filipescu,
ca banul Costache Ghica și alții, se gândeau la ce era să zică
de dânșii împăratul Alexandru al Rusiei sau împăratul
Leopold al Austriei când or afla că au trimis soli la desculții
de francezi, nimeni nu îndrăznea să iasă din vorba banuluiboierimii, dară inima lui îl trăgea mai mult către lumea
învățată, către profesori, către oamenii de litere și către

# Conclusions

We implemented a RAG system for Romanian. The LLM used was a quantized version of the Romanian LLM.

Using RAG, we can focus the answers on the exact set of documents we are targeting. We could see that, for some of the questions, we get very "to the point" answers in the case of RAG system while using the direct prompting the LLM method we get also some halucinations.

The answers to the questions are delivered in relatively short time, around 2-3 seconds most of them. The quality of the responses is rather good.

I suggest to you to download a copy of the Ion Ghica's work and check few of the facts in the text. 

And, of course, you can try to index some other works by a famous Romanian writer.

